In [35]:
#pip install python-dateutil

In [1]:
import json
import codecs
from dateutil.parser import parse

In [2]:
!ls

K8S.ipynb                  firestore.ipynb
create_jsons.ipynb         google_cloud_storage.ipynb
data_20200405.jsonline     to_del.ipynb


In [28]:
with codecs.open("data_20200405.jsonline","r") as file:
    jsonline_flats = file.readlines()

In [29]:
def json2firestore(rec):
    tmp = json.loads(rec)
    _ = tmp.pop('body')
    tmp['download_date'] = parse(tmp['download_date']) if tmp['download_date'] else None
    tmp['date_created'] = parse(tmp['date_created']) if tmp['date_created'] else None
    tmp['date_modified'] = parse(tmp['date_modified']) if tmp['date_modified'] else None
    return tmp
    
jsonline_flats = [json2firestore(i) for i in jsonline_flats]

In [3]:
from google.cloud import firestore

import tarfile
import json
import codecs
import os
from tqdm.notebook import tqdm_notebook
from dateutil import parser

In [4]:
!pwd

/Users/xszpo/GoogleDrive/01_Projects/202003_xFlats_K8S/gcp


In [5]:
root_path = "/Users/xszpo/GoogleDrive/01_Projects/202003_xFlats_K8S/secrets/"
gcp_secrets = "firestore.json"
secters_path = os.path.join(root_path, gcp_secrets)

In [6]:
db = firestore.Client.from_service_account_json(secters_path)

In [59]:
def firestore_write(json, collection_name='flats'):
    return db.collection(collection_name).document(json['_id']).set(json)


In [62]:
firestore_write({"_id":"test","test":"test"})

update_time {
  seconds: 1586117741
  nanos: 45762000
}

In [37]:
for i in jsonline_flats:
    firestore_write(i)

In [7]:
def check_if_exists(_id, collection_name='flats'):
    return db.collection(collection_name).document(_id).get([]).exists

In [6]:
users_ref = db.collection(u'users')
docs = users_ref.stream()

for doc in docs:
    print(u'{} => {}'.format(doc.id, doc.to_dict()))


In [73]:
with codecs.open('data/olx/olx_316139968_430000.json','r') as file:
    json_ = json.load(file)

_id = json_.pop('_id')
flat_size = json_.pop('flat_size', None)
rooms = json_.pop('rooms', None)
floor = json_.pop('floor', None)
building_type = json_.pop('building_type', None)
year_of_building = json_.pop('year_of_building', None)
property_form = json_.pop('property_form',None)
price = json_.pop('price', None)
prediction = json_.pop('prediction', None)
producer_name = json_.pop('producer_name', None)
price_m2 = json_.pop('price_m2', None)
download_date = parser.parse(json_.pop('download_date'))
GC_latitude = json_.pop('GC_latitude', None)
GC_longitude = json_.pop('GC_longitude', None)
GC_addr_neighbourhood = json_.pop('GC_addr_neighbourhood', None)
GC_addr_suburb = json_.pop('GC_addr_suburb', None)
GC_addr_road = json_.pop('GC_addr_road', None)
GC_addr_city = json_.pop('GC_addr_city', None)
GC_addr_postcode = json_.pop('GC_addr_postcode', None)
download_date_utc = json_.pop('download_date_utc', None)

json_1 = {
    '_id': _id,
    'flat_size': flat_size,
    'rooms': rooms,
    'floor': floor,
    'building_type': building_type,
    'year_of_building': year_of_building,
    'property_form': property_form,
    'price': price,
    'prediction': prediction,
    'producer_name': producer_name,
    'price_m2': price_m2,
    'download_date': download_date,
    'GC_latitude': GC_latitude,
    'GC_longitude': GC_longitude,
    'GC_addr_neighbourhood':GC_addr_neighbourhood, 
    'GC_addr_suburb': GC_addr_suburb,
    'GC_addr_road': GC_addr_road,
    'GC_addr_city': GC_addr_city,
    'GC_addr_postcode': GC_addr_postcode,
    'download_date_utc': download_date_utc
}

json_2 = json_


In [74]:
producer = json_1['producer_name']
name = json_1['_id']

doc_ref1 = db.collection(producer).document(name)
doc_ref1.set(json_1)

doc_ref2 = doc_ref1.collection('more').document('data')
doc_ref2.set(json_2)

update_time {
  seconds: 1584917212
  nanos: 912316000
}

In [18]:
#! ls data/olx

In [72]:
json__ = doc_ref1.get().to_dict()
json__.update(doc_ref2.get().to_dict())

json__

{'_id': 'olx_316139968_430000',
 'GC_addr_road': 'Zakręcona',
 'building_type': 'Blok',
 'GC_addr_neighbourhood': 'Białołęka Dworska',
 'GC_addr_postcode': '03-030',
 'GC_addr_city': 'Warszawa',
 'GC_latitude': 52.33008,
 'producer_name': 'olx',
 'property_form': None,
 'download_date': DatetimeWithNanoseconds(2019, 11, 23, 5, 3, 20, 989000, tzinfo=<UTC>),
 'flat_size': 53,
 'price_m2': 8113.21,
 'price': 430000,
 'rooms': 2,
 'year_of_building': None,
 'GC_addr_suburb': 'Białołęka',
 'download_date_utc': 1574481800.989,
 'GC_longitude': 20.99661,
 'prediction': None,
 'floor': 7,
 'location': 'Warszawa, Mazowieckie, Białołęka',
 'url': 'https://www.olx.pl/oferta/sprzedam-bezposrednio-mieszkanie-dwupokojowe-na-bialolece-CID3-IDlourE.html',
 'main_url': 'https://www.olx.pl/nieruchomosci/mieszkania/sprzedaz/warszawa',
 'name': 'Sprzedam bezpośrednio mieszkanie dwupokojowe na Białołęce.',
 'GC_addr_country': 'Polska',
 'GC_boundingbox': '52.3300261,52.3302106,20.996558,20.9968113',
 'GC_a